In [17]:
from datetime import datetime as dt
import csv


class Order:

    # Class variables
    product_file = "products.csv"
    log_file = "log.txt"
    discount_rate = 0

    def __init__(self):
        self.items = []

    @staticmethod
    def _get_time():
        
        """Return current timestamp as string"""
        return dt.now().strftime("%Y-%m-%d %H:%M:%S")

    @classmethod
    def _log(cls, message):
        
        """Write a message to log.txt with time stamp."""
        with open(cls.log_file, "a") as file:
            file.write(f"[{cls._get_time()}] {message}\n")

    @staticmethod
    def log_action(func):
        """Decorator to log method execution"""

        def wrapper(self, *args, **kwargs):
            Order._log(f"Executed {func.__name__}")
            
            return func(self, *args, **kwargs)

        return wrapper

    @log_action
    def add_item_by_id(self, product_id, quantity):
        
        """Adds a product by its ID from CSV"""
        with open(self.product_file, newline="") as file:
            reader = csv.DictReader(file)
           
            for row in reader:
                if row["id"] == str(product_id):
                    
                    name = row["name"]
                    price = float(row["price"])
                    total = price * quantity
                   
                    self.items.append((name, price, quantity, total))
                    self._log(f"Added item: {name} (x{quantity}) - Total: {total}")
                    return
        
        self._log(f"Invalid product ID attempt: {product_id}")

    @classmethod
    def set_discount(cls, rate):
      
        """Set the discount rate for all orders"""
        cls.discount_rate = rate
        cls._log(f"Set discount rate to {rate*100}%")

    @log_action
    def calculate_total(self):
        
        """Calculate total after discount"""
        
        subtotal = sum(item[3] for item in self.items)
        discount_amount = subtotal * (self.discount_rate / 100)
        final_total = subtotal - discount_amount
        
        self._log(
            f"Calculated total: Subtotal={subtotal}, Discount={discount_amount}, Final={final_total}"
        )
        return final_total

    @log_action
    def show_order(self):
        
        """Print order summary"""
        if not self.items:
            print("Your order is empty.")
            return
        
        print("\n----- Order Summary -----")
        
        for name, price, quantity, total in self.items:
            print(f"{name} - {price} x {quantity} = {total}")
        
        subtotal = sum(item[3] for item in self.items)
        discount_amount = subtotal * (self.discount_rate / 100)
        final_total = subtotal - discount_amount
        
        print("--------------------------")
        print(f"Subtotal: {subtotal}")
        print(f"Discount: {discount_amount}")
        print(f"Final Total: {final_total}")
        print("--------------------------")
        self._log("Order summary shown")

In [ ]:
# Create an order object
order1 = Order()

# Add items (valid)
order1.add_item_by_id(1, 2)  # Laptop x2
order1.add_item_by_id(4, 3)  # Mouse x3
order1.add_item_by_id(2, 2)  
order1.add_item_by_id(7, 6)  

# Add item (invalid ID)
order1.add_item_by_id(99, 1)  # Invalid

# Apply discount
Order.set_discount(30)

# Calculate total
final_total = order1.calculate_total()
print(f"Final Total: {final_total}")

print("Actions completed. Check log.txt for details.")

Final Total: 1764.0
Actions completed. Check log.txt for details.
